In [10]:
pip install geemap

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.1/256.1 KB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 25.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.9/230.9 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━

In [18]:
import folium
import xarray as xr
import numpy as np
import ipywidgets as widgets
from folium.plugins import HeatMap
from IPython.display import display

# Load the NetCDF file
netcdf_url = "/home/mohamed/Documents/ERA5/Data/t2m_20180106.nc"
ds = xr.open_dataset(netcdf_url)
t2m = ds["t2m"].values

# Create a Folium Map
m = folium.Map(location=[lat.mean(), lon.mean()], zoom_start=5)

# Get the latitude and longitude arrays
lat = ds["latitude"].values
lon = ds["longitude"].values

heatmaps = []
t2m_layer = folium.FeatureGroup(name='2m Temperature')  # Create a FeatureGroup for t2m data

def update_heatmap(time_step):
    if heatmaps:
        for heatmap in heatmaps:
            m.remove_child(heatmap)
        heatmaps.clear()

    heatmap_data = list(zip(lat, lon, t2m[time_step]))
    folium_heatmap = HeatMap(heatmap_data, radius=5, gradient={0.2: "blue", 0.4: "green", 0.6: "yellow", 1.0: "red"})
    m.add_child(folium_heatmap)
    heatmaps.append(folium_heatmap)

time_slider = widgets.IntSlider(min=0, max=t2m.shape[0] - 1, description="Time Step")
widgets.interactive(update_heatmap, time_step=time_slider)

# Add the t2m layer to the FeatureGroup
t2m_layer.add_to(m)

folium.LayerControl().add_to(m)

display(m)
